In [1]:
import pandas as pd
import numpy as np
from scipy import stats
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
import json
from IPython.display import display
from subprocess import check_output
import sys
import traceback
from tqdm.notebook import tqdm, trange
import plotly.graph_objects as go
import plotly.express as px
pd.options.mode.chained_assignment = None
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
techs = {
    "tech1": "1q_1o",
    "tech2": "k_per_terra",
    "tech3": "4pip",
    "tech4": "7vp",
    "tech5": "1o_1pw",
    "tech6": "1k_1c",
    "tech7": "3vp_per_gaia_place",
    "tech8": "4c",
    "tech9": "4pw",
    # idk wtf this is --> "tech-ship0"
}

adv_techs = {
    "advtech1": "3vp_per_fed_pass",
    "advtech2": "2vp_per_tech_bump",
    "advtech3": "1qic_5c_action",
    "advtech4": "2vp_per_mine",
    "advtech5": "3vp_per_rl_pass",
    "advtech6": "1o_per_sector",
    "advtech7": "1vp_per_terra_pass",
    "advtech8": "2vp_per_gaia",
    "advtech9": "4vp_per_ts",
    "advtech10": "2vp_per_sector",
    "advtech11": "3o_action",
    "advtech12": "5vp_per_fed",
    "advtech13": "3k_action",
    "advtech14": "3vp_per_mine_place",
    "advtech15": "3vp_per_ts_place",
}

feds = {
    "fed1": "12vp",
    "fed2": "qic",
    "fed3": "2pw",
    "fed4": "2o",
    "fed5": "6c",
    "fed6": "2k",
    "gleens": "gleens"
}

round_scorings = {
    "score1": "2vp_per_terra",
    "score2": "2vp_per_research_bump",
    "score3": "2vp_per_mine_place",
    "score4": "5vp_per_fed_place",
    "score5": "4vp_per_ts_place",
    "score6": "4vp_per_gaia_place",
    "score7": "5vp_per_3pip_place",
    "score8": "3vp_per_ts_place",
    "score9": "3vp_per_gaia_place",
    "score10": "5vp_per_3pip_place",
}

boosters = {

    "booster1": "1k_1o",
    "booster2": "2pwt_1o",
    "booster3": "1qic_2c",
    "booster4": "2c_terra",
    "booster5": "2pw_nav",
    "booster6": "1o_1vp_per_mine",
    "booster7": "1o_2vp_per_ts",
    "booster8": "1k_3vp_per_rl",
    "booster9": "4pw_4vp_per_3pip",
    "booster10": "4c_1vp_per_gaia",
}

bad_buildings = {"colony", 
                 "colonyShip", 
                 "tradeShip", 
                 "constructionShip", 
                 "researchShip",
                 "scout",
                 "frigate",
                 "battleShip",
                 "customsPost",
                 "tradePost"
                }

factions = {
    "terrans",
    "lantids",
    "hadsch-hallas",
    "ivits",
    "baltaks",
    "geodens",
    "xenos",
    "gleens",
    "ambas",
    "taklons",
    "bescods",
    "firaks",
    "itars",
    "nevlas"
}

In [3]:
dat_2 = pd.read_pickle("two_players_data")
dat_3 = pd.read_pickle("three_players_data")
dat_4 = pd.read_pickle("four_players_data")

In [4]:
dat_4['pos_1_dropped'].describe()

count     10982
unique        2
top       False
freq      10952
Name: pos_1_dropped, dtype: object

In [7]:
for faction in factions:
    factions_pos = []
    for pos in range(4):
        dat = dat_4
        
        # select faction
        dat = dat[dat["pos_" + str(pos + 1) +"_faction"] == faction]
        
        #remove dropped players
        dat = dat[dat['pos_' + str(pos + 1) + '_dropped'] == False]
        
        dat['pos_' + str(pos + 1) + '_score'] += -1 * dat['pos_' + str(pos + 1) + '_score_bid']
        
        #labels
        prefix = "pos_" + str(pos + 1) + '_buildings_r_1'
        structs_class = 'r1_structs_class_' + faction
        structs_exact = 'r1_structs_exact_' + faction
        over_all_score = 'ave_score_' + faction
        over_all_elo = 'ave_elo_' + faction
        
        #get scores and elos
        dat[over_all_score] = dat['pos_' + str(pos + 1) + '_score']
        dat[over_all_elo] = dat['pos_' + str(pos + 1) + '_elo']
        
        #structs placed
        dat[prefix + '_PI'] = pd.to_numeric(dat[prefix + '_PI'], downcast='integer')
        dat[prefix + '_ac1'] = pd.to_numeric(dat[prefix + '_ac1'], downcast='integer')
        dat[prefix + '_ac2'] = pd.to_numeric(dat[prefix + '_ac2'], downcast='integer')
        dat[prefix + '_lab'] = pd.to_numeric(dat[prefix + '_lab'], downcast='integer')
        dat[prefix + '_ts'] = pd.to_numeric(dat[prefix + '_ts'], downcast='integer')
        dat[prefix + '_m'] = pd.to_numeric(dat[prefix + '_m'], downcast='integer')
        
        # create general structs catergory                        
        dat.loc[dat[prefix + '_PI'] > 0, structs_class] = 'PI'
        dat.loc[dat[prefix + '_ac1'] > 0, structs_class] = 'ac1'
        dat.loc[dat[prefix + '_ac2'] > 0, structs_class] = 'ac2'
        dat.loc[
            (dat[prefix + '_lab'] > 0) &
            (dat[prefix + '_PI'] == 0) &
            (dat[prefix + '_ac1'] == 0) &
            (dat[prefix + '_ac2'] == 0)
            , structs_class] = 'lab'
        
        dat.loc[
            (dat[prefix + '_ts'] > 0) &
            (dat[prefix + '_lab'] == 0) &
            (dat[prefix + '_PI'] == 0) &
            (dat[prefix + '_ac1'] == 0) &
            (dat[prefix + '_ac2'] == 0)
            , structs_class] = 'ts'
        
        dat.loc[
            (dat[prefix + '_ts'] == 0) &
            (dat[prefix + '_lab'] == 0) &
            (dat[prefix + '_PI'] == 0) &
            (dat[prefix + '_ac1'] == 0) &
            (dat[prefix + '_ac2'] == 0)
            , structs_class] = 'm'
        
        
        #create exact category
        dat[structs_exact] = [" " +
            str(m) + '-m_' +
            str(ts) + '-ts_' +
            str(lab) + '-lab_' + 
            str(PI) + '-pi_' +
            str(ac1) + '-ac1_' +
            str(ac2) + '-ac2' 
            for m, ts, lab, PI, ac1, ac2 in zip(
            dat[prefix + '_m'], 
            dat[prefix + '_ts'], 
            dat[prefix + '_lab'],
            dat[prefix + '_PI'], 
            dat[prefix + '_ac1'], 
            dat[prefix + '_ac2'])
        ]
        
        
        factions_pos.append(dat)
        
    factions_in = pd.concat(factions_pos, ignore_index=True)
    
    unique_starts = factions_in[[structs_exact, structs_class, over_all_score, over_all_elo]]
    unique_starts['class_counts'] = 0
    counts = unique_starts.groupby(structs_exact)['class_counts'].transform('count').to_frame()
    unique_starts = unique_starts[[structs_exact, structs_class, over_all_score, over_all_elo]]
    unique_starts = pd.concat([unique_starts, counts], axis=1)
    unique_starts = unique_starts.groupby([structs_class, structs_exact]).mean().reset_index()
    unique_starts['class_counts'] = pd.to_numeric(unique_starts['class_counts'], downcast='integer')
    
    #remove really low scores
    #unique_starts = unique_starts[(np.abs(stats.zscore(unique_starts[over_all_score])) > 2)]
    unique_starts = unique_starts[unique_starts[over_all_score].between(unique_starts[over_all_score].quantile(.15), unique_starts[over_all_score].quantile(1))]
    
    
    #average score overall for position played till end
    average_overall = (np.mean(dat_4[dat_4['pos_1_dropped'] == False]['pos_1_score']) + 
                      np.mean(dat_4[dat_4['pos_2_dropped'] == False]['pos_2_score']) + 
                      np.mean(dat_4[dat_4['pos_3_dropped'] == False]['pos_3_score']) + 
                      np.mean(dat_4[dat_4['pos_4_dropped'] == False]['pos_4_score'])) / 4
    
    #midpoint is average for all factions overall - but color ends are highest and lowest score by that faction
    fig = px.treemap(unique_starts, path=[px.Constant('R1 Structs'), structs_class, structs_exact], values='class_counts',
                  color=over_all_score, hover_data=[over_all_elo],
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=(average_overall))
    fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
    
    fig.update_layout(title_text="R1 Structures: Frequency and Impact on Score Relative to All Factions Average (" + str(round(average_overall, 2)) + "): " + faction)
    
    fig.write_html("./plots/r1_structs_analysis/freq_and_score/" + faction + ".html")
    fig.show()
    
    """
    fig = px.sunburst(unique_starts, path=[px.Constant('R1 Structures: Frequency and Overall Score for ' + faction), structs_class, structs_exact], values='class_counts',
                  color=over_all_score, hover_data=[over_all_elo],
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.median(unique_starts[over_all_score]))
    fig.show()
    """
    
    
    
    

In [6]:
# checking out if data wrong for unlikely taklon start --> it happend!
p = 'pos_1_buildings_r_1'
taklons_1 = dat_4[dat_4['pos_1_faction'] == 'taklons']
taklons_1 = taklons_1.loc[(taklons_1[p + '_m'] == 7) & (taklons_1[p + '_lab'] == 1)]
taklons_1.head()

,id,map_layout,num_players,1k_1o,2pwt_1o,1qic_2c,2c_terra,2pw_nav,1o_1vp_per_mine,1o_2vp_per_ts,1k_3vp_per_rl,4pw_4vp_per_3pip,4c_1vp_per_gaia,tech_terra,tech_nav,tech_int,tech_gaia,tech_eco,tech_sci,tech_free1,tech_free2,tech_free3,tech_adv-terra,tech_adv-nav,tech_adv-int,tech_adv-gaia,tech_adv-eco,tech_adv-sci,round_1_scoring,round_2_scoring,round_3_scoring,round_4_scoring,round_5_scoring,round_6_scoring,final_scoring_1,final_scoring_2,pos_4_dropped,pos_4_elo,pos_4_faction,pos_4_score,pos_4_start_pos,pos_4_bid,pos_4_feds_taken,pos_4_fed_12vp,pos_4_fed_qic,pos_4_fed_2pw,pos_4_fed_2o,pos_4_fed_6c,pos_4_fed_2k,pos_4_fed_gleens,...,pos_2_adv_tech_taken_1qic_5c_action,pos_2_adv_tech_taken_2vp_per_mine,pos_2_adv_tech_taken_3vp_per_rl_pass,pos_2_adv_tech_taken_1o_per_sector,pos_2_adv_tech_taken_1vp_per_terra_pass,pos_2_adv_tech_taken_2vp_per_gaia,pos_2_adv_tech_taken_4vp_per_ts,pos_2_adv_tech_taken_2vp_per_sector,pos_2_adv_tech_taken_3o_action,pos_2_adv_tech_taken_5vp_per_fed,pos_2_adv_tech_taken_3k_action,pos_2_adv_tech_taken_3vp_per_mine_place,pos_2_adv_tech_taken_3vp_per_ts_place,pos_2_total_techs_taken,pos_2_buildings_r_1_m,pos_2_buildings_r_1_ts,pos_2_buildings_r_1_lab,pos_2_buildings_r_1_ac1,pos_2_buildings_r_1_ac2,pos_2_buildings_r_1_PI,pos_2_buildings_r_1_gf,pos_2_buildings_r_2_m,pos_2_buildings_r_2_ts,pos_2_buildings_r_2_lab,pos_2_buildings_r_2_ac1,pos_2_buildings_r_2_ac2,pos_2_buildings_r_2_PI,pos_2_buildings_r_2_gf,pos_2_buildings_r_3_m,pos_2_buildings_r_3_ts,pos_2_buildings_r_3_lab,pos_2_buildings_r_3_ac1,pos_2_buildings_r_3_ac2,pos_2_buildings_r_3_PI,pos_2_buildings_r_3_gf,pos_2_buildings_r_4_m,pos_2_buildings_r_4_ts,pos_2_buildings_r_4_lab,pos_2_buildings_r_4_ac1,pos_2_buildings_r_4_ac2,pos_2_buildings_r_4_PI,pos_2_buildings_r_4_gf,pos_2_buildings_r_5_m,pos_2_buildings_r_5_ts,pos_2_buildings_r_5_lab,pos_2_buildings_r_5_ac1,pos_2_buildings_r_5_ac2,pos_2_buildings_r_5_PI,pos_2_buildings_r_5_gf,average_elo
4364,Imperial-tree-6131,standard,4,False,True,True,False,True,True,True,True,False,True,k_per_terra,4pw,3vp_per_gaia_place,4c,1k_1c,1q_1o,4pip,1o_1pw,7vp,3k_action,1o_per_sector,1vp_per_terra_pass,2vp_per_gaia,2vp_per_tech_bump,2vp_per_mine,2vp_per_terra,5vp_per_fed_place,4vp_per_gaia_place,4vp_per_ts_place,5vp_per_3pip_place,3vp_per_ts_place,gaia,structureFed,False,100.0,bescods,126.0,3.0,0.0,3.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,False,True,False,False,False,False,False,False,False,False,False,False,False,7.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,6.0,2.0,0.0,0.0,0.0,1.0,0.0,7.0,0.0,1.0,1.0,0.0,1.0,0.0,6.0,1.0,2.0,1.0,0.0,1.0,0.0,8.0,1.0,1.0,1.0,1.0,1.0,0.0,177.75
